In [1]:
#https://www.kaggle.com/pujakarki/nlp-the-simplest-way

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from nltk.stem import WordNetLemmatizer

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


In [2]:
train_full = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
train = train_full[['text', 'target']]
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [3]:
# stem each word in the given text
def lemmatize(text):    
    text = [wnl.lemmatize(word) for word in text.split()]
    return " ".join(text)
wnl = WordNetLemmatizer()
train['text'] = train['text'].apply(lemmatize)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [4]:
vectorizer = CountVectorizer(analyzer='word', binary=True)
vectorizer.fit(train['text'])
X = vectorizer.transform(train['text']).todense()
y = train['target'].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
# predict on the section of the train data
y_pred = logreg.predict(X_test)
f1score = f1_score(y_test, y_pred)
print(f"Model Score: {f1score}")

Model Score: 0.7659574468085106


In [7]:
# vectorize test data and predict for submission
test_X = vectorizer.transform(test['text']).todense()
submission['target'] = logreg.predict(test_X)
submission.to_csv("submission.csv", index=False)
submission.head()

,id,target
0,0,1
1,2,0
2,3,1
3,9,0
4,11,1
